# WebScrapping
Vamos a realizar un webscrapping de un directorio de empresas

Dado que la estructura de navegación del directorio es similar a un arbol, iremos analizando y extrayendo los datos

a medida que necesitemos desde la raiz del arbol hasta las hojas. Las hojas sería los datos de cada compañia



En nuestra tabla inicial pretendemos sacar una seríe de datos, que son los productos, compradores y vendedores
```htm
<div id="categorytable">
    <table>
        <tbody>
            <tr>
                <td>
                <ul class="ul_home_products">
                    <li><a href="abanico/">ABANICO</a>&nbsp;<span class="home_sellers"><a href="abanico/?groupService=46">1</a></span>&nbsp;<span class="home_buyers"><a href="abanico/?groupService=45">0</a></span></li>
                    <li><a href="albaricoques/">ALBARICOQUES</a>&nbsp;<span class="home_sellers"><a href="albaricoques/?groupService=46">29</a></span>&nbsp;<span class="home_buyers"><a href="albaricoques/?groupService=45">7</a></span></li>
                </ul>
                </td>
                <td>
                <ul class="ul_home_products">
                    <li><a href="datiles/">DATILES</a>&nbsp;<span class="home_sellers"><a href="datiles/?groupService=46">18</a></span>&nbsp;<span class="home_buyers"><a href="datiles/?groupService=45">4</a></span></li>
                    <li><a href="endrinas/">ENDRINAS</a>&nbsp;<span class="home_sellers"><a href="endrinas/?groupService=46">0</a></span>&nbsp;<span class="home_buyers"><a href="endrinas/?groupService=45">0</a></span></li>
                </ul>
                </td>
                <td>
                <ul class="ul_home_products">
                    <li><a href="limones/">LIMONES</a>&nbsp;<span class="home_sellers"><a href="limones/?groupService=46">38</a></span>&nbsp;<span class="home_buyers"><a href="limones/?groupService=45">13</a></span></li>
                    <li><a href="litchis/">LITCHIS</a>&nbsp;<span class="home_sellers"><a href="litchis/?groupService=46">4</a></span>&nbsp;<span class="home_buyers"><a href="litchis/?groupService=45">1</a></span></li>
                </ul>
                </td>
                <td>
                <ul class="ul_home_products">
                    <li><a href="naranjas/">NARANJAS</a>&nbsp;<span class="home_sellers"><a href="naranjas/?groupService=46">46</a></span>&nbsp;<span class="home_buyers"><a href="naranjas/?groupService=45">13</a></span></li>
                    <li><a href="nectarinas/">NECTARINAS</a>&nbsp;<span class="home_sellers"><a href="nectarinas/?groupService=46">16</a></span>&nbsp;<span class="home_buyers"><a href="nectarinas/?groupService=45">2</a></span></li>
                </ul>
                </td>
            </tr>
        </tbody>
    </table>
</div>
```

El codigo html donde debemos extraer los datos para completar nuestra tabla de inicio es: 
```html
<li>
    <a href="abanico/">ABANICO</a>
    <span class="home_sellers">
        <a href="abanico/?groupService=46">1</a>
    </span>
    <span class="home_buyers">
        <a href="abanico/?groupService=45">0</a>
    </span>
</li>
```



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de la página
url = "https://www.organic-bio.com/es/directorio/frutos/"

# Realizar la solicitud HTTP
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Encontrar la tabla de categorías
category_table = soup.find("div", id="categorytable")

# Lista para almacenar los datos
products = []

# Base URL para completar enlaces
base_url = "https://www.organic-bio.com/es/directorio/frutos/"

# Extraer los productos
if category_table:
    for li in category_table.find_all("li"):
        product_name = li.find("a").text.strip()

        # Buscar enlaces de vendedores y compradores de forma segura
        seller_span = li.find("span", class_="home_sellers")
        buyer_span = li.find("span", class_="home_buyers")

        seller_link = seller_span.find("a")["href"] if seller_span and seller_span.find("a") else None
        buyer_link = buyer_span.find("a")["href"] if buyer_span and buyer_span.find("a") else None

        #Como el atributo href solo tiene la direccion relativa, debemos añadir url base para completar la dirección url completa
        full_seller_url = base_url + seller_link if seller_link else "No disponible"
        full_buyer_url = base_url + buyer_link if buyer_link else "No disponible"

        #Vamos añadiendo en la lista de productos un diccionario con los tres campos que necesitamos
        # Producto, Vendedores y Compradores
        products.append({
            "Producto": product_name,
            "Vendedores": full_seller_url,
            "Compradores": full_buyer_url
        })
print(f'Lista de productos: {products}')
# Guardar en Excel
df = pd.DataFrame(products)
#print(df)
df.to_excel("productos.xlsx", index=False, sheet_name="Productos")

print("Datos extraídos y guardados en productos2.xlsx")



A partir de una tabla que hemos creado en la que tenemos productos y los enlaces a la categoria de vendedores y compradores
de ese determinado producto, recorreremos todos estos enlaces para ir obteniendo el directorio completo de los datos de las empresas.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Leer la tabla de productos
productos_df = pd.read_excel("productos.xlsx")

# Base URL
base_url = "https://www.organic-bio.com/es/"

def obtener_paginas(url):
    """Obtiene todas las páginas de un listado paginado"""
    paginas = [url]
    print(f'La pagina donde se tienen que obtener los enlaces a las diversas paginas es:\n{url}')
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    pages_div = soup.find("div", id="pages")
    if pages_div and pages_div.find_all("a"):
        # Vamos a guardar las paginas en un conjunto para evitar los duplicados,
        # ya que el enlace a siguente es un enlace duplicado en el <div id="pages">
        paginas=set()
        url_base=url.split("?")[0]
        for a in pages_div.find_all("a"):
            page_link = a["href"]
            if page_link not in paginas:
                paginas.add(url_base + page_link)
    print(f'Paginas: {paginas}')
    return paginas



def obtener_empresas(url):
    """Obtiene los enlaces a las empresas desde una página donde esta el listado de estas"""
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al acceder a {url}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    empresas = []
    
    # Buscar la tabla con id="firms"
    tabla = soup.find('table', {'id': 'firms'})
    if not tabla:
        print("No se encontró la tabla de empresas en la página.")
        return []
    
    # Recorrer las filas de la tabla
    for fila in tabla.find_all('tr')[1:]:  # Saltamos la primera fila que es la cabecera
        celda = fila.find('td')  # Primera celda de la fila
        if celda:
            enlace = celda.find('a')
            if enlace and 'href' in enlace.attrs:
                empresas.append(enlace['href'])
    print(f'Empresas: {empresas}')
    return empresas


def extraer_datos_empresa(url):
    
    '''Recibe un enlace donde se encuentra la tabla donde están los datos de la empresa
        y a partir de este enlace y la tabla obtiene todos los datos que se almacenarán en un diccionario
        con las keys como los campos de la tabla y los values como los valores de los campos'''
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Error al acceder a {url}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    tabla = soup.find('table', {'id': 'company'})
    
    if not tabla:
        print("No se encontró la tabla de la empresa en la página.")
        return None
    
    datos_empresa = {}
    
    for fila in tabla.find_all('tr'):
        columnas = fila.find_all(['th', 'td'])
        if len(columnas) == 2:
            campo = columnas[0].text.strip()
            valor = columnas[1].text.strip()
            datos_empresa[campo] = valor
    
    return datos_empresa

# Lista para almacenar los datos finales
empresas_datos = []

# Recorrer la tabla de productos
for index, row in productos_df.iterrows():
    #print(f'Index: {index} y Row: {row["Vendedores"]}')
    producto = row["Producto"]
    for categoria, url in [("Vendedor", row["Vendedores"]), ("Comprador", row["Compradores"])]:
        print(f'Producto: {producto} \nCategoria: {categoria}\nUrl: {url}')
        paginas=obtener_paginas(url)
        
        if url != "No disponible":
            #Vamos a recorrer el listado completo buscando en todas las paginas
            # de el producto y la categoria en cuestion
            for pagina in paginas:
                empresas = obtener_empresas(pagina)
                for empresa_url in empresas:
                    datos_empresa = extraer_datos_empresa(empresa_url)
                    # Guardamos los productos y la categoría a la que pertenece la empresa en los datos de la empresa
                    datos_empresa["Producto"] = producto
                    datos_empresa["Categoría"] = categoria
                    #Añadimos la empresa al listado de empresas
                    empresas_datos.append(datos_empresa)
                    time.sleep(0.1)  # Para evitar bloqueos

# Guardar en un nuevo Excel
df_final = pd.DataFrame(empresas_datos)
#df_final.to_excel("empresas.xlsx", index=False, sheet_name="Empresas")

print("Proceso completado. Datos guardados en empresas2.xlsx")